In [3]:
import requests
import numpy as np
from lxml import etree
from datetime import  date,datetime, timedelta
import xml.etree.ElementTree as ET
from operator import itemgetter

In [58]:
torneos = ["Champions","Copa America", "Copa Paraguay", "Eliminatorias","España","Italia","Libertadores","Paraguay","Premier League","Recopa","Sudamericana"]
torneovec =["champions","copaamerica", "copaparaguay", "eliminatorias","espana","italia","libertadores","paraguay","premierleague","recopa","sudamericana"]
estadisticas = ["posiciones","fixture","jugadores.estadisticas"]
estadistica_top = {
    "Goleadores" : "Goals",
    "Asistencias": "GoalAssistances",
    "Recuperacion" : "Stealing",
    "Faltas Cometidas" : 'Fouls',
    "Faltas Recibidas" : 'FoulsOpponents',
    "Tarjetas Amarillas" : 'YellowCards',
    "Tarjetas Rojas" : 'RedCards'
}
x='-1111'
while int(x) < 0  or int(x) > 10:
    print("Seleccionar torneo:")
    for i in range(len(torneos)):
        print(str(i) + "- " + torneos[i])
    x = input()
torneo = torneovec[int(x)]

Seleccionar torneo:
0- Champions
1- Copa America
2- Copa Paraguay
3- Eliminatorias
4- España
5- Italia
6- Libertadores
7- Paraguay
8- Premier League
9- Recopa
10- Sudamericana
8


In [92]:
#funciones
estadisticas_dict = {}
def traer_datos(torneo,estadistica):
    url = f'https://editorialdenegocios.com/datafactory/xml/es/{torneo}/deportes.futbol.{torneo}.{estadistica}.xml'
    print(url)
    web = requests.get(url)
    root = ET.fromstring(web.text)
    return root
def posiciones():
    posiciones = np.array([[0,-1,0,0,0,0,0,0,0]],dtype=object)
    if torneo != 'sudamericana':
        root = traer_datos(torneo,estadisticas[0])
        if torneo == 'champions' or torneo == 'libertadores':#ordenar tabla solo para champions y libertadores
            root[:] = sorted(root, key=lambda child: (child.tag,child.get('zona')))
            for c in root:
                c[:] = sorted(c, key=lambda child: (child.tag,child.get('puntos')))
            
        f.write('Tabla de posiciones ; PJ ; G ; E ; P ; GF ; GC ; DF ; Pts;\n')
        contador=0
        for i in root.findall('equipo'):
            if contador % 4 == 0 and (torneo == 'champions' or torneo == 'libertadores' or torneo == 'sudamericana'):
                f.write(i.get("zona") + "\n")
            text = [i.find('nombre').text, i.find('jugados').text, i.find('ganados').text, i.find('empatados').text, i.find('perdidos').text,i.find('golesfavor').text, i.find('golescontra').text, i.find('difgol').text, i.find('puntos').text]
            posiciones = np.vstack([posiciones,np.asarray(text,object)]) 
            contador = contador + 1
              
    estadisticas_dict.update({"Posiciones":posiciones[1:]})
def fixture():
    fecha_pasada = np.array([[0,-1,0,0]],dtype=object)
    fecha_actual_jugada = np.array([[0,-1,0,0,0]],dtype=object)
    fecha_actual = np.array([[0,-1,0,0,0]],dtype=object)
    root = traer_datos(torneo,estadisticas[1])
    for i in root.findall("fecha"):
        if(i.get('estado')=='ultima'):
            for j in i:
                if(j.find('local')!= None):
                    text = [j.find("local").text,j.find("goleslocal").text,j.find("golesvisitante").text,j.find("visitante").text]
                    fecha_pasada = np.vstack([fecha_pasada,np.asarray(text,object)]) 
        if(i.get('estado')=='actual'):
            for j in i:
                if(j.find('local')!= None):
                    if (j.find('estado').attrib['id'] == '2'):
                        text = [j.find("local").text,j.find("goleslocal").text,j.find("golesvisitante").text,j.find("visitante").text,j.get("tipo")]
                        fecha_actual_jugada = np.vstack([fecha_actual_jugada,np.asarray(text,object)]) 
                    else:
                        text = [j.find("local").text, j.find("visitante").text, j.get("tipo"),f'{j.get("fecha")[6:8]}/{j.get("fecha")[4:6]}/{j.get("fecha")[0:4]}',j.get("hora")]
                        fecha_actual = np.vstack([fecha_actual,np.asarray(text,object)]) 
    estadisticas_dict.update({"Fecha Pasada": fecha_pasada[1:]})
    estadisticas_dict.update({"Fecha Actual Jugadas": fecha_actual_jugada[1:]})
    estadisticas_dict.update({"Fecha Actual a Jugar": fecha_actual[1:]})
def cargar_vector_top(estadistica):
    if estadistica != "Pases":
        vector_top = np.array([0,-1,0],dtype=object)
    else:
        vector_top = np.array([0,-1,0,0, 0],dtype=object)
    for i in root.findall('Tournament/Players/Player'):
        if estadistica != "Pases":
            text = [i.attrib['name'] + " " + i.attrib['lastName'], int(i.find(estadistica_top[estadistica]).attrib['total']) , i.attrib['currentTeam']]
        else:
            text = [i.attrib['name'] + " " + i.attrib['lastName'], int(i.find('Passes').attrib['successful']) , int(i.find('Passes').attrib['total']), float(i.find('Passes').attrib['effectiveness'])  , i.attrib['currentTeam']]
        vector_top = np.vstack([vector_top,np.asarray(text,object)]) 
    f.write(f'\n {estadistica} \n')
    if estadistica != 'Pases':
        vector_top = sorted(vector_top, key=lambda a_entry: a_entry[1], reverse=True)
    else: 
        vector_top = sorted(vector_top, key=lambda a_entry: a_entry[2], reverse=True)
        pases10 = np.array([0,-1,0,0, 0],dtype=object)
        for i in range(11):
            pases10 = np.vstack([pases10,vector_top[i]])  
            pases10[i][3] = int(pases10[i][3])
    
        pases10 = sorted(pases10, key=lambda a_entry: a_entry[3], reverse=True)
        vector_top = pases10

    
    estadisticas_dict.update({estadistica: vector_top[0:10]})   

In [ ]:
titulocsv = f'datos{torneo}.csv'
f = open(titulocsv,'w')
posiciones()
fixture()
root = traer_datos(torneo,estadisticas[2])
cargar_vector_top("Goleadores")
cargar_vector_top("Asistencias")
cargar_vector_top("Pases")
cargar_vector_top("Recuperacion")
cargar_vector_top("Faltas Cometidas")
cargar_vector_top("Faltas Recibidas")
cargar_vector_top("Tarjetas Amarillas")
cargar_vector_top("Tarjetas Rojas")
f.close()

https://editorialdenegocios.com/datafactory/xml/es/premierleague/deportes.futbol.premierleague.posiciones.xml


In [95]:
estadisticas_dict["Posiciones"]

array([['Tottenham', '12', '7', '4', '1', '24', '10', '14', '25'],
       ['Liverpool', '12', '7', '4', '1', '27', '18', '9', '25'],
       ['Southampton', '13', '7', '3', '3', '25', '18', '7', '24'],
       ['Leicester City', '13', '8', '0', '5', '24', '17', '7', '24'],
       ['Everton', '13', '7', '2', '4', '23', '18', '5', '23'],
       ['Chelsea', '13', '6', '4', '3', '26', '14', '12', '22'],
       ['Manchester City', '12', '5', '5', '2', '18', '12', '6', '20'],
       ['West Ham United', '12', '6', '2', '4', '20', '15', '5', '20'],
       ['Manchester United', '11', '6', '2', '3', '19', '17', '2', '20'],
       ['Wolverhampton', '13', '6', '2', '5', '13', '17', '-4', '20'],
       ['Aston Villa', '10', '6', '0', '4', '21', '13', '8', '18'],
       ['Crystal Palace', '12', '5', '2', '5', '18', '17', '1', '17'],
       ['Leeds United', '13', '5', '2', '6', '22', '24', '-2', '17'],
       ['Newcastle United', '12', '5', '2', '5', '16', '21', '-5', '17'],
       ['Arsenal', '13', '4